In [1]:
import numpy as np
import pandas as pd
import os

# Clean lab mono Gnorm Ed data

In [2]:
df = pd.read_csv(os.path.join('data', 'lab_mono_Gnorm_Ed.csv'))
# split the data into N dataframes, every 2 columns
new_dfs = {}
# and save them to separate files
for i in range(0, df.shape[1], 2):
    # get the column names
    col1 = df.columns[i]
    col2 = df.columns[i + 1]
    # create a new dataframe with the two columns
    new_df = df[[col1, col2]]
    test_id = new_df.columns[0].split("_")[0]
    new_df.columns = ["Ed_prc", "Gtan/fe/fp"]
    new_dfs[test_id] = new_df

In [7]:
# stack the dataframes and add a new column with the test_id
stacked_df = pd.concat(new_dfs.values(), keys=new_dfs.keys())
stacked_df = stacked_df.reset_index(level=0)
# drop nans
stacked_df = stacked_df.dropna()
# rename the columns
stacked_df.columns = ["test_id", "Ed_prc", "Gtan/fe/fp"]
# save the dataframe to a csv file
stacked_df.to_csv(os.path.join('data', 'lab_mono_Gnorm_Ed_long.csv'), index=False)

# Clean raw CPT qc and fs data

In [23]:
df = pd.read_csv(os.path.join('data', 'cpt_raw_data.csv'))
# stack the data from second column onwards
# use the first column as index
df = df.set_index(df.columns[0])
df = df.stack().reset_index()
df = df.dropna()
df.head()
# split second column strings by "_"
df['data_location_id'] = df.iloc[:, 1].str.split("_").str[0]
df['value_type'] = df.iloc[:, 1].str.split("_").str[1]
df['unit'] = df.iloc[:, 1].str.split("_").str[2]
# drop the second column
df = df.drop(df.columns[1], axis=1)
# rename the first 2 columns
df.columns = ["Depth_m", "value", "data_location_id", "value_type", "unit"]
# reorder the columns
df = df[["data_location_id", "Depth_m", "value_type", "unit", "value"]]
# sort df by data_location_id, Depth_m then value_type
df = df.sort_values(by=["data_location_id", "Depth_m", "value_type"])
# save the dataframe to a csv file
df.to_csv(os.path.join('data', 'cpt_raw_data_long.csv'), index=False)

# Clean CPT correlations (static)

In [5]:
df = pd.read_csv(os.path.join('data', 'cpt_corr_Gmax_examples_static.csv'))

# stack the data from second column onwards
# use the first column as index
df = df.set_index(df.columns[0])
df = df.stack().reset_index()
# remove nan values
df = df.dropna()
# rename columns
df.columns = ['Depth_m', 'Correlation_method', 'Gmax_MPa']
# convert depth to float
df['Depth_m'] = df['Depth_m'].astype(float)
# convert Gmax to float
df['Gmax_MPa'] = df['Gmax_MPa'].astype(float)
# convert correlation method to string
df['Correlation_method'] = df['Correlation_method'].astype(str)
# remove leading and trailing spaces
df['Correlation_method'] = df['Correlation_method'].str.strip()
# write out to csv
df.to_csv(os.path.join('data', 'cpt_corr_Gmax_examples_static_long.csv'), index=False)